## Switching Bitrate

Finding bitrates at which switching should happen to a different frame-rate

#### Importing Libaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import CubicHermiteSpline, CubicSpline
import math

import os,pathlib,sys,warnings
warnings.filterwarnings('ignore')
sys.path.append("/home/krishnasrikardurbha/Desktop/Dynamic-Frame-Rate")
from tqdm import tqdm
import json
import functions.software_commands as software_commands
import functions.statistics as statistics
import defaults

### Stored Videos

In [2]:
# Path
stored_videos_path = "/home/krishnasrikardurbha/Desktop/Dynamic-Frame-Rate/dataset/stored_videos"

File2Metadata = {}

for resolution in [(1920,1080), (1280,720), (960,540)]:
	resolution_string = "{}x{}".format(resolution[0], resolution[1])
	File2Metadata[resolution_string] = {}
	
	for fps in [30,20,10]:
		File2Metadata[resolution_string][int(fps)] = {}
		
		for bitrate in reversed([3, 2.75, 2.5, 2.25, 2, 1.75, 1.5, 1.25, 1, 0.75]):
			File2Metadata[resolution_string][int(fps)][bitrate] = {}

			for video_file in os.listdir(stored_videos_path):
				File2Metadata[resolution_string][int(fps)][bitrate][os.path.splitext(video_file)[0]] = {}


# Information to Mode
Info2Mode = {}
Mode2Info = {}
for i,complexity in enumerate([(False, False), (True, False), (False, True), (True, True)]):
	Info2Mode[complexity] = i
	Mode2Info[i] = complexity

### Simulated Compressed Videos

In [3]:
# Paths
compressed_videos_segments_dir = "/home/krishnasrikardurbha/Desktop/Dynamic-Frame-Rate/dataset/compressed_video_segments"

# Compressed videos for different resolution, fps and bitrates
for resolution in [(1920,1080), (1280,720), (960,540)]:
	resolution_string = "{}x{}".format(resolution[0], resolution[1])
	
	for fps in [30,20,10]:
		for bitrate in [3, 2.75, 2.5, 2.25, 2, 1.75, 1.5, 1.25, 1, 0.75]:
			# For each compressed video
			compressed_videos_setting_dir = os.path.join(compressed_videos_segments_dir, resolution_string, str(int(fps)), str(bitrate))
			quality_dir = os.path.join(defaults.quality_scores, resolution_string, str(int(fps)), str(bitrate))
			
			for video_file in os.listdir(compressed_videos_setting_dir):	
				filemetadata = os.path.splitext(video_file)[0].split("_")
				original_filename = "_".join(filemetadata[:2])
				part = int(filemetadata[-1])
				mode = int(filemetadata[0][-1])
				
				resolution, fps, b, q = statistics.get_statistics(
					video_path=os.path.join(compressed_videos_setting_dir, video_file),
					quality_dir=quality_dir
				)

				scene_complexity, vehicle_info = Mode2Info[mode]

				File2Metadata[resolution_string][int(fps)][bitrate][original_filename][part] = [b, q]


			# Rate-Quality Information for a resolution, fps and bitrate
			for key in File2Metadata[resolution_string][int(fps)][bitrate].keys():
				data = File2Metadata[resolution_string][int(fps)][bitrate][key]

				data = dict(sorted(data.items()))

				metadata = np.asarray(list(data.values()))
				q = np.mean(metadata[:, 1])
				b = np.mean(metadata[:, 0])

				File2Metadata[resolution_string][int(fps)][bitrate][key] = [b, q]
				
		
		# Rate-Quality Information for a resolution and fps
		Data = File2Metadata[resolution_string][int(fps)]
		File2Metadata[resolution_string][int(fps)] = {}

		for key in Data[3].keys():
			Bitrate = []
			Quality = []
			File2Metadata[resolution_string][int(fps)][key] = []

			for bitrate in reversed([3, 2.75, 2.5, 2.25, 2, 1.75, 1.5, 1.25, 1, 0.75]):
				metadata = Data[bitrate][key]
				File2Metadata[resolution_string][int(fps)][key].append([metadata[0], metadata[1]])

### Finding Switching Frame-Rate

In [4]:
def dydx(x, y, return_size="same"):
	"""
	Returns slopes by calculating dy/dx i.e (y2-y1)/(x2-x1)
	Args:
		x (list/np.array): Values of x
		y (list/np.array): Values of y
		return_size (str): Options: ["same", "valid"], return size of slopes. If set to "valid", returns slopes of length len(x)-1 slope values. Else returns slopes of size len(x) by appending the last value again.
	"""
	den = np.diff(x)
	num = np.diff(y)

	slope = list(np.divide(num,den+1e-8))
	if return_size == "same":
		slope.append(slope[-1])
	else:
		None

	# Handling Nans in slope
	for i in range(len(slope)):
		if math.isnan(slope[i]):
			slope[i] = slope[i-1]
	
	slope = np.asarray(slope)
	
	# Assertion
	assert (return_size == "same") and (slope.shape[0] == y.shape[0]), "dy/dx and y do not have same shape."

	return slope


def Find_Intersection(
	f:np.array=None,
	g:np.array=None,
	x:np.array=None,
	use_interpolation:bool=True
):
	"""
	Finding intersection between two functions f and g for given values of x using Cubic-Hermite Interpolation.
	Args:
		f (np.array): 2D numpy array with f[:,0] containing x values and f[:,1] containing output of the function. (Default: None)
		g (np.array): 2D numpy array with f[:,0] containing x values and f[:,1] containing output of the function. (Default: None)
		x (np.array): Values of x for which the intersection point needs to be found. (Default: None)
		use_interpolation (bool): Whether or not to use interpolation. (Default: True)
	Returns:
		f_new (np.array): Interpolated values of function "f" for values of x.
		g_new (np.array): Interpolated values of function "g" for values of x.
		(bool): True if f > g after last intersection point or if f > g when there is no intersection point. False if f < g for the cases mentioned.
	"""
	if use_interpolation == True:
		# Checking if Inputs are in increasing sequence
		# if np.all(np.diff(f[:,0]) >= 0) and np.all(np.diff(f[:,1]) >= 0) and np.all(np.diff(g[:,0]) >= 0) and np.all(np.diff(g[:,1]) >= 0):
		# 	None
		# else:
		# 	print (f[:,0], f[:,1])
		# 	assert False, "Non-Increasing Sequence Found!!!"

		f_Function = CubicSpline(f[:,0], f[:,1])
		g_Function = CubicSpline(g[:,0], g[:,1])

		f_new = np.round(f_Function(x), decimals=3)
		g_new = np.round(g_Function(x), decimals=3)
	else:
		assert np.array_equal(f[:,0], g[:,0]), "Interpolation not used. The 'x' values of functions 'f' and 'g' do not match."
		f_new = f[:,1]
		g_new = g[:,1]
	
	sign = np.sign(f_new - g_new)
	diff = np.diff(sign)
	idx = np.argwhere(diff).flatten()

	if idx.shape[0] == 0:
		if np.all(sign[-5:,] >= 0):
			# No-intersection and f > g for given values of x
			return (np.asarray(f_new),np.asarray(g_new), True, [])
		else:
			# No-intersection and f < g for given values of x
			return (np.asarray(f_new),np.asarray(g_new), False, [])
	else:
		if diff[idx[-1]] > 0:
			# Intersection and after final intesection point idx[-1], f > g for given values of x.
			return (np.asarray(f_new),np.asarray(g_new), True, [idx[-1]])
		else:
			# Intersection and after final intesection point idx[-1], f < g for given values of x.
			return (np.asarray(f_new),np.asarray(g_new), False, [idx[-1]])
		

def find_switching_framerate(
	RQ_pairs:dict,
	frame_rates:list=[30,20,10]
):
	crossover_bitrates = np.inf*np.ones((len(frame_rates)-1,))
	for i in range(len(frame_rates)-1):
		f = RQ_pairs[frame_rates[i]]
		g = RQ_pairs[frame_rates[i+1]]

		assert frame_rates[i] >= frame_rates[i+1] and frame_rates[i] >= frame_rates[i+1], "Wrong order of framerates"

		if f.shape[0] <= 1 or g.shape[0] <= 1:
			continue

		if np.min(f[:,0]) >= np.max(g[:,0]):
			# Doesn't have a common bitrate range, then the lowest bitrate of higher resolution is considered as cross-over bitrate
			crossover_bitrates[i] = np.min(f[:,0])
		else:			
			# start and end shows bitrate range of intersection between two framerates
			b_start = max(np.min(f[:,0]),np.min(g[:,0]))
			b_end = min(np.max(f[:,0]),np.max(g[:,0]))
			num_points = 50*np.ceil(b_end-b_start).astype(int)
			x = np.round(np.linspace(start=b_start, stop=b_end, num=num_points, endpoint=True), decimals=4)

			results = Find_Intersection(f,g,x)
			# print (results[2], len(results[3]), x[results[3]])
			if results[2] == True and len(results[3]) > 0:
				# If after the final intersection, higher resolution has higher quality than lower framerates, considering intersection bitrate as cross-over bitrate.
				crossover_bitrates[i] = x[results[3]]
			elif results[2] == False and len(results[3]) > 0:
				# If after the final intersection, lower resolution has higher quality than higher framerates, considering highest bitrate where overlap ends is considered as cross-over bitrate.
				crossover_bitrates[i] = b_end
			elif results[2] == False and len(results[3]) == 0:
				# If no intersection is found between lower and higher framerates and lowest resolution dominates, considering highest bitrate where overlap ends as cross-over bitrate.
				crossover_bitrates[i] = b_end
			else:
				# If no intersection is found between lower and higher framerates and highest resolution dominates, considering lowest bitrate where overlap starts as cross-over bitrate.
				crossover_bitrates[i] = b_start

			assert crossover_bitrates[i] >= b_start and crossover_bitrates[i] <= b_end, "The values of cross-over bitrate did not lie in the intersection bitrate region."

	# Rounding Bitrates
	crossover_bitrates = np.round(crossover_bitrates, decimals=4)

	# Imposing Monotonicity on estimated cross-over bitrates
	for i in range(1,len(crossover_bitrates)):
		crossover_bitrates[i] = min(crossover_bitrates[i], crossover_bitrates[i-1])

	return crossover_bitrates

### Calculating Switching-Bitrates

In [5]:
# Setting
scene_complexity = False
vehicle_info = False

for filenum in [1,2,3]:
    original_filename = "mode{}_{}".format(Info2Mode[(scene_complexity, vehicle_info)], filenum)
    print (original_filename)

    RQ_pairs = {}
    for fps in [30,20,10]:
        RQ_pairs[fps] = np.round(np.asarray(File2Metadata["1920x1080"][fps][original_filename]), decimals=3)

    switching_bitrates = find_switching_framerate(RQ_pairs=RQ_pairs, frame_rates=[30,20,10])
    print (switching_bitrates)

mode0_1
[2.7663 2.7663]
mode0_2
[3.013 3.003]
mode0_3
[2.984 2.984]


In [6]:
# Setting
scene_complexity = False
vehicle_info = True

for filenum in [1,2,3]:
    original_filename = "mode{}_{}".format(Info2Mode[(scene_complexity, vehicle_info)], filenum)
    print (original_filename)

    RQ_pairs = {}
    for fps in [30,20,10]:
        RQ_pairs[fps] = np.round(np.asarray(File2Metadata["1920x1080"][fps][original_filename]), decimals=3)

    switching_bitrates = find_switching_framerate(RQ_pairs=RQ_pairs, frame_rates=[30,20,10])
    print (switching_bitrates)

mode2_1
[3.012  2.1188]
mode2_2
[2.2118 1.6508]
mode2_3
[2.0248 1.222 ]
